#Introducing Decorators

A decorator is a function that is applied to another function (or a class).
Functions can, of course, be passed as arguments to other functions.

In [ ]:
def ff(f, a):
    return f(a)

In [ ]:
ff(len, "12345")

In [ ]:
ff(sum, [1, 2, 3, 4])

Functions can also _return_ a function.
Here the `add_to()` function defines an internal function and returns that.

In [ ]:
def add_to(n):
    def adder(x):
        return n+x
    return adder

In [ ]:
add_to(6)

In [ ]:
fa = add_to(6)

In [ ]:
fa(3)

In [ ]:
add_to(10)(7)

The `string_required()` function takes a function as its argument and returns
a function that will raise an exception if its argument is not a string.
Otherwise it applies the function to the argument and returns the function's result.

In [ ]:
def string_required(f):
    def string_checked(arg):
        if type(arg) != str:
            raise ValueError(repr(arg)+" is not a string")
        return f(arg)
    return string_checked

The `must_have_s()` function just prints its argument and the argument's length.

In [ ]:
def must_have_s(arg):
    print(arg, len(arg))

In [ ]:
must_have_s("steve")

In [ ]:
must_have_s([1, 2, 3])

As you can see, however, it doesn't live up to its name, as it will accept non-string arguments.
So now we'll modify it to require a string argument.

In [ ]:
must_have_s = string_required(must_have_s)

Python has a specific syntax to apply the _decorator_ at the time the function is defined.

In [ ]:
@string_required
def second(s):
    return 3*s

In [ ]:
second("steve")

In [ ]:
second((1, 2, 3))

If the same decorator function is applied to a class, the class's `__init__()` method will require a string argument.

In [ ]:
@string_required
class little:
    def __init__(self, s):
        self.s = s

In [ ]:
person = little("steve")

In [ ]:
person

In [ ]:
person2 = little([1, 2, 3])

In [ ]:
person.s

The following class decorator function takes a class and attempts to wrap all its
(non-magic) methods so they print "Woof" before they run.

In [ ]:
def barking(cls):
    for name in cls.__dict__:
        if name.startswith("__"):
            continue
        func = getattr(cls, name)
        def woofer(*args, **kw):
            print("Woof")
            return func(*args, **kw)
        setattr(cls, name, woofer)
    return cls

This seems to work for a simple class.

In [ ]:
@barking
class dog_1:
    def shout(self):
        print("hello from dog_1")

In [ ]:
d1 = dog_1()

In [ ]:
d1

In [ ]:
d1.shout()

When the class has multiple methods, however, a problem becomes apparent.

In [ ]:
@barking
class dog_3(dog_1):
    def wag(self):
        print("a dog_3 is happy")
    def sniff(self):
        print("a dog_3 is curious")

In [ ]:
d3 = dog_3()

In [ ]:
d3.wag(); d3.sniff(); d3.shout()

Notice that the new class's `wag()` and `sniff()` methods now appear to do the same thing.
This is a complex problem to do with closures and cells (I wrote a blog post about it).

We can fix this by using a different closure, replacing it with a default
argument to a wrapper function.

In [ ]:
def barking(cls):
    for name in cls.__dict__:
        if name.startswith("__"):
            continue
        func = getattr(cls, name)
        def wrapper(func=func):
            def woofer(*args, **kw):
                print("Woof")
                return func(*args, **kw)
            return woofer
        setattr(cls, name, wrapper(func))
    return cls

In [ ]:
@barking
class dog_3(dog_1):
    def wag(self):
        print("a dog_3 is happy")
    def sniff(self):
        print("a dog_3 is curious")

In [ ]:
d3 = dog_3()
d3.wag()
d3.sniff()
d3.shout()

Just to prove that decorators don't _have_ to make sense, here's a slightly mad example.

In [ ]:
def completely_barking(cls):
    class Null:
        pass
    return Null

In [ ]:
@completely_barking
class something:
    def meth1(self):
        pass
    def meth2(self):
        pass

In [ ]:
something

In [ ]:
s = something()
something.meth1()